In [284]:
username = 'psharma' 
password = 'Vado345###' 
caseid = '6492'
FundedDate = '4/12/2022'
CuredBy = 48
CuredDate = '3/12/2020'
TicketRequestorname = 'John Akindude'
Ticket = '24281193'
qsf_template = r"C:\Users\psharma\Desktop\New folder\QSF Funded Template.xlsx"

In [286]:
import pyodbc
import os
import pandas as pd
import glob
import datetime as dt 
import xlwings as xw
server = 'reportdb01.archer.local' 
database = 'S3Reporting'
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
wb = xw.Book(qsf_template)
clientquery = (
'select Id, LastName, FirstName, StatusId, ClientFunded, FundedDate \n'
'from Clientsviews \n'
'where CaseId in ('+caseid+')')
Lienquery = (
'select Clientid , Id, Stage, Lientype, StatusId \n'
'from fullproductviews \n'
'where CaseId in ('+caseid+')')

defiquery = (
'Select DeficiencyId, EntityId, EntityTypeId, EntityTypeName, DeficiencyTypeId, TypeName, StageId, StageName, Note, CreatedBy, CreatedByName,CuredBy, CuredDate, f.ClientId \n'
'from Deficiencies d \n'
'Full Outer Join FullProductViews f on d.EntityId = f.ClientId \n'
'where EntityId in (select ClientId from fullproductviews where CaseId in ('+caseid+')) \n' 
'and EntityTypeId = 4 and stageid!=4 \n'
'UNION \n'
'select DeficiencyId, EntityId, EntityTypeId, EntityTypeName, DeficiencyTypeId, TypeName, StageId, StageName, Note, CreatedBy, CreatedByName,CuredBy, CuredDate, f.ClientId \n'
'from Deficiencies d \n'
'Full Outer Join FullProductViews f on d.EntityId = f.Id \n'
'where EntityId in (select Id from fullproductviews where CaseId in ('+caseid+')) \n' 
'and EntityTypeId = 1 and stageid!=4 \n')
clientdf = pd.read_sql_query(clientquery, cnxn)
liendf = pd.read_sql_query(Lienquery, cnxn)
defidf = pd.read_sql_query(defiquery, cnxn)

clientsql_pre = wb.sheets['QSF client update_SQL (pre)']
clientsql_post = wb.sheets['QSF client update_SQL (post)']
liensql_pre = wb.sheets['QSF lien update_SQL (pre)']
liensql_post = wb.sheets['QSF lien update_SQL (post)']
defisql_pre = wb.sheets['Deficiency SQL(pre)']
defisql_post = wb.sheets['Deficiency SQL(post)']

clientsql_pre.range('A1').options(pd.DataFrame, index=False).value = clientdf
clientsql_pre.range('i1').value = clientquery
clientsql_pre.range('i1').api.WrapText = False
clientsql_post.range('i1').value = clientquery
clientsql_post.range('i1').api.WrapText = False
liensql_pre.range('A1').options(pd.DataFrame, index=False).value = liendf
liensql_pre.range('i1').value = Lienquery
liensql_pre.range('i1').api.WrapText = False
liensql_post.range('i1').value = Lienquery
liensql_post.range('i1').api.WrapText = False

defisql_pre.range('A1').options(pd.DataFrame, index=False).value = defidf
defisql_pre.range('x1').value = defiquery
defisql_pre.range('x1').api.WrapText = False
defisql_post.range('x1').value = defiquery
defisql_post.range('x1').api.WrapText = False

client_df = clientdf.query('ClientFunded !="True"')
client_df['ClientFunded'] = client_df['ClientFunded'].replace([False,None],'True')
client_df.loc[:,'ClientFunded'] = True
client_df.loc[:,'FundedDate'] = FundedDate
client_update = wb.sheets['Client Update']
client_update.range('A1').options(pd.DataFrame, index=False).value = client_df[['Id','ClientFunded','FundedDate']]

lien_df = liendf.query('Lientype == "Medicare Lien - MT" and Stage== "Pending Internal Finalization"').replace(['Pending Internal Finalization'],'To Send - Final Demand')

lienntnote = (
'Lien Updated from Pending Internal Finalization  to To Send - Final Demand '
' as per instructions given by '+TicketRequestorname+' on ticket #'+Ticket
)
Lien_update = lien_df[['Id','Stage']]
Lien_update['NewClientNote']=lienntnote
Lien_update


Lien_update = wb.sheets['Lien Update']
Lien_update.range('A1').options(pd.DataFrame, index=False).value = lien_df[['Id','Stage']]

defi_df = defidf.query('Note == "Client Funded Status" or Note == "QSF Funded date"')
defi_df.loc[:,'CuredBy'] = CuredBy
defi_df.loc[:,'CuredDate'] = CuredDate
defi_update = wb.sheets['Deficiency Update']
defi_update.range('A1').options(pd.DataFrame, index=False).value = defi_df[['DeficiencyId','StageId','CuredBy','CuredDate']]
wb.save()

,Id,Stage,NewClientNote
